### 참조 사이트 : https://github.com/googleapis/python-aiplatform/blob/main/samples/snippets/prediction_service/predict_image_classification_sample.py

In [47]:
IMAGE_PATH = "/home/jupyter/cat.jpg"

In [72]:
import base64

from google.cloud import aiplatform
from google.cloud.aiplatform.gapic.schema import predict

def predict_image_classification_sample(
    project: str,
    endpoint_id: str,
    filename: str,
    location: str = "us-central1",
    api_endpoint: str = "us-central1-aiplatform.googleapis.com",
):
    # The AI Platform services require regional API endpoints.
    client_options = {"api_endpoint": api_endpoint}
    # Initialize client that will be used to create and send requests.
    # This client only needs to be created once, and can be reused for multiple requests.
    client = aiplatform.gapic.PredictionServiceClient(client_options=client_options)
    with open(filename, "rb") as f:
        file_content = f.read()

    # The format of each instance should conform to the deployed model's prediction input schema.
    encoded_content = base64.b64encode(file_content).decode("utf-8")
    instance = predict.instance.ImageClassificationPredictionInstance(
        content=encoded_content,
    ).to_value()
    instances = [instance]
    # See gs://google-cloud-aiplatform/schema/predict/params/image_classification_1.0.0.yaml for the format of the parameters.
    parameters = predict.params.ImageClassificationPredictionParams(
        confidence_threshold=0.5,
        max_predictions=5,
    ).to_value()
    endpoint = client.endpoint_path(
        project=project, location=location, endpoint=endpoint_id
    )
    response = client.predict(
        endpoint=endpoint, instances=instances, parameters=parameters
    )
    print("response")
    print(" deployed_model_id:", response.deployed_model_id)
    # See gs://google-cloud-aiplatform/schema/predict/prediction/image_classification_1.0.0.yaml for the format of the predictions.
    predictions = response.predictions
    for prediction in predictions:
        print(" prediction:", dict(prediction))


In [73]:
predict_image_classification_sample(
    project="959298622983",
    endpoint_id="4467836912165453824",
    location="us-central1",
    filename=IMAGE_PATH
)

response
 deployed_model_id: 913252159007490048
 prediction: {'confidences': [0.951833844], 'ids': ['1557212205017989120'], 'displayNames': ['cat']}


In [36]:
#

## 일괄 예측

In [59]:
import json
with open('test-data.json', 'w') as fp:
    json.dump(x_test, fp)

!gsutil cp test-data.json gs://kaggle-data-upload

Copying file://test-data.json [Content-Type=application/json]...
/ [1 files][286.6 KiB/286.6 KiB]                                                
Operation completed over 1 objects/286.6 KiB.                                    


In [60]:
my_model=aiplatform.Model("projects/959298622983/locations/us-central1/models/3539563225299288064")

In [ ]:
batch_prediction_job = my_model.batch_predict(
    job_display_name='dog-cat_batch_predict',
    gcs_source='gs://kaggle-data/test-data.json',
    gcs_destination_prefix='gs://{YOUR_BUCKET}/prediction-results',
    machine_type='n1-standard-4',)